In [3]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from dataclasses import dataclass
from scipy.optimize import curve_fit
import sympy as sympy

In [4]:
@dataclass
class soilDict:
    K_s     : float
    theta_s : float
    theta_r : float
    alpha   : float
    nExpon  : float
    mExpon  : float = 0
    
    def __post_init__(self):
        self.mExpon = 1.0 - (1.0/self.nExpon)


In [13]:
theta,theta_s,theta_r,alpha,h,n,m = sympy.symbols("theta theta_s theta_r alpha h n m")

In [20]:
expr = (theta_s - theta_r) * (1 + (-alpha*h)**n)**-m + theta_r

In [22]:
theta_e = sympy.symbols("theta_e")
theta_e = (1 + (-alpha*h)**n)**-m
theta_e

((-alpha*h)**n + 1)**(-m)

In [23]:
expr

theta_r + (-theta_r + theta_s)/((-alpha*h)**n + 1)**m

In [24]:
sympy.diff(expr,h)

-m*n*(-alpha*h)**n*(-theta_r + theta_s)/(h*((-alpha*h)**n + 1)*((-alpha*h)**n + 1)**m)

In [ ]:
soil = soilDict(2.89E-6,0.43,0.078,3.6,1.56)

def thetaEffectiveCalc(h,soil): 
    se = np.where(
            h >= 0,
            1.0,
            np.power(1.0 + np.power(- soil.alpha * h, soil.nExpon), - soil.mExpon)
            )
    return se

def refactorthetaEffectiveCalc(h,soil): 
    se = np.power(1.0 + np.power(np.maximum(-soil.alpha * h, 0), soil.nExpon), - soil.mExpon)
    return se

def thetaCalc(h,soil:soilDict):
    theta = thetaEffectiveCalc(h,soil) * (soil.theta_s - soil.theta_r) + soil.theta_r   
    return theta
    
def capillaryCalc(h,soil:soilDict):
    capillary = np.where(
                    h >= 0, 
                    0.0,
                    (soil.theta_s - soil.theta_r) * soil.mExpon * soil.nExpon * soil.alpha * np.power(-soil.alpha * h, soil.nExpon - 1.0) * np.power(1.0 + np.power(-soil.alpha * h, soil.nExpon), - soil.mExpon - 1.0)
                    )
    return capillary

def hydraulicCondCalc(h,soil):
    se = thetaEffectiveCalc(h,soil)
    kr = np.sqrt(se) * np.power(1 - np.power(1 - np.power(se, 1/soil.mExpon),soil.mExpon),2)
    return kr

In [ ]:
hLin = np.linspace(-5,2,200)
thetaLin = thetaEffectiveCalc(hLin,soil=soil)
thetaLin2 = refactorthetaEffectiveCalc(hLin,soil=soil)

In [ ]:
fig,axs = plt.subplots(1,1,figsize=[8,8])
ax = axs
ax.plot(hLin,thetaLin)
#ax.plot(hLin,thetaLin2)
#ax.axhline(y=soil.theta_s)
#ax.axhline(y=soil.theta_r)
ax.set(ylabel=r'$\theta$',xlabel=r'$h$')

# ax = axs[1]
# ax.plot(hLin,capiLin)

# ax = axs[2]
# ax.plot(hLin,permLin)
plt.show()